<a href="https://colab.research.google.com/github/sharon-kathambi/data-segmentation-with-unet/blob/main/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import gc
import tensorflow as tf

gc.collect()
tf.keras.backend.clear_session()

Split Dataset into Training and Test

In [1]:
import os
import shutil
import random

# Define paths
source_folder = "/content/drive/MyDrive/Dataset_BUSI_with_GT"
train_folder = "data/traindataset"
test_folder = "data/testdataset"

# Define categories
categories = ["benign", "malignant", "normal"]

# Create train and test directories with category subfolders
for category in categories:
    os.makedirs(os.path.join(train_folder, category), exist_ok=True)
    os.makedirs(os.path.join(test_folder, category), exist_ok=True)

# Function to move images and masks
def move_image_and_mask(image_list, src_category_path, dest_category_path):
    for img in image_list:
        img_path = os.path.join(src_category_path, img)
        mask_path = os.path.join(src_category_path, img.replace('.png', '_mask.png'))

        # Copy image
        shutil.copy2(img_path, os.path.join(dest_category_path, img))

        # Copy mask if it exists
        if os.path.exists(mask_path):
            shutil.copy2(mask_path, os.path.join(dest_category_path, os.path.basename(mask_path)))

# Process each category
for category in categories:
    category_path = os.path.join(source_folder, category)
    train_category_path = os.path.join(train_folder, category)
    test_category_path = os.path.join(test_folder, category)

    # Get all images (excluding masks)
    all_images = [f for f in os.listdir(category_path) if f.endswith(('.png')) and not f.endswith('_mask.png')]

    # Shuffle images randomly
    random.shuffle(all_images)

    # Split 80% for training and 20% for testing
    split_index = int(0.85 * len(all_images))
    train_images = all_images[:split_index]
    test_images = all_images[split_index:]

    # Move images and masks
    move_image_and_mask(train_images, category_path, train_category_path)
    move_image_and_mask(test_images, category_path, test_category_path)

    print(f"{category}: Training images = {len(train_images)}, Testing images = {len(test_images)}")

print("Dataset split completed successfully!")



benign: Training images = 385, Testing images = 69
malignant: Training images = 179, Testing images = 32
normal: Training images = 113, Testing images = 20
Dataset split completed successfully!


Loading Data

In [2]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split

# Image parameters
IMG_HEIGHT = 512
IMG_WIDTH = 512
IMG_CHANNELS = 3  # Color images (RGB)

def load_dataset(root_dir):
    categories = ["benign", "malignant", "normal"]
    images = []
    masks = []

    for category in categories:
        folder_path = os.path.join(root_dir, category)

        for filename in os.listdir(folder_path):
            if filename.endswith(".png") and "_mask" not in filename:
                # Load the color image (RGB)
                img_path = os.path.join(folder_path, filename)
                img = load_img(img_path, color_mode="rgb", target_size=(IMG_HEIGHT, IMG_WIDTH))
                img = img_to_array(img) / 255.0  # Normalize to [0, 1]

                # Load the corresponding mask (grayscale)
                mask_filename = filename.replace(".png", "_mask.png")
                mask_path = os.path.join(folder_path, mask_filename)

                if os.path.exists(mask_path):
                    mask = load_img(mask_path, color_mode="grayscale", target_size=(IMG_HEIGHT, IMG_WIDTH))
                    mask = img_to_array(mask) / 255.0
                    mask = (mask > 0.5).astype(np.float32)  # Binary mask

                    images.append(img)
                    masks.append(mask)
                else:
                    print(f"Warning: No mask found for {filename}")

    return np.array(images), np.array(masks)

# Load the dataset from Google Drive
dataset_path = "/content/data/traindataset"
X, Y = load_dataset(dataset_path)

# Split into training (85%) and validation (15%)
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.15, random_state=42)

# Normalize images
#X_train = X_train.astype('float32') / 255.0
#X_val = X_val.astype('float32') / 255.0

# Normalize and reshape masks
#Y_train = (Y_train.astype('float32') / 255.0).squeeze()
#Y_val = (Y_val.astype('float32') / 255.0).squeeze()

#Y_train = np.expand_dims(Y_train, axis=-1)
#Y_val = np.expand_dims(Y_val, axis=-1)

# Print shapes for verification
print(f"X_train shape: {X_train.shape}, Y_train shape: {Y_train.shape}")
print(f"X_val shape: {X_val.shape}, Y_val shape: {Y_val.shape}")


print(f"Training: {len(X_train)} images, Validation: {len(X_val)} images")
print("X_train shape:", X_train.shape)
print("Y_train shape:", Y_train.shape)
print("Y_val shape:", Y_val.shape)


X_train shape: (563, 512, 512, 3), Y_train shape: (563, 512, 512, 1)
X_val shape: (100, 512, 512, 3), Y_val shape: (100, 512, 512, 1)
Training: 563 images, Validation: 100 images
X_train shape: (563, 512, 512, 3)
Y_train shape: (563, 512, 512, 1)
Y_val shape: (100, 512, 512, 1)


In [3]:
!pip install keras-unet


Train model

In [4]:
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from keras_unet.models import custom_unet
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K
from sklearn.metrics import precision_score, recall_score, f1_score

# Set parameters
IMG_HEIGHT = 512
IMG_WIDTH = 512
IMG_CHANNELS = 3  # RGB images
BATCH_SIZE = 8
INITIAL_LR = 0.0001
EPOCHS = 50

# Learning rate scheduler (alternative to StepLR)
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=INITIAL_LR,
    decay_steps=0.5 * (len(X_train) // BATCH_SIZE),
    decay_rate=0.96,
    staircase=True
)

# Define Dice Coefficient & Loss
def dice_coefficient(y_true, y_pred, smooth=1):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def dice_loss(y_true, y_pred):
    return 1 - dice_coefficient(y_true, y_pred)

# Build U-Net Model
model = custom_unet(input_shape=(512, 512, 3),
                    filters=32, use_batch_norm=True,
                    num_classes=1, activation="sigmoid")

# Compile Model
model.compile(optimizer=Adam(learning_rate=lr_schedule),
              loss=dice_loss,
              metrics=[dice_coefficient])

# Data Augmentation
data_gen_args = dict(rotation_range=15,
                     width_shift_range=0.1,
                     height_shift_range=0.1,
                     zoom_range=0.2,
                     horizontal_flip=True,
                     fill_mode="nearest")

image_datagen = ImageDataGenerator(**data_gen_args)
mask_datagen = ImageDataGenerator(**data_gen_args)

image_datagen.fit(X_train, augment=True)
mask_datagen.fit(Y_train, augment=True)

image_generator = image_datagen.flow(X_train, batch_size=BATCH_SIZE, seed=42)
mask_generator = mask_datagen.flow(Y_train, batch_size=BATCH_SIZE, seed=42)

def data_generator(image_gen, mask_gen):
    while True:
        img_batch = next(image_gen)[0]
        mask_batch = next(mask_gen)[0]

        # Ensure masks have the correct shape (BATCH_SIZE, 512, 512, 1)
        if len(mask_batch.shape) == 3:
            mask_batch = np.expand_dims(mask_batch, axis=-1)

        yield img_batch, mask_batch

train_dataset = tf.data.Dataset.from_generator(
    lambda: data_generator(image_generator, mask_generator),
    output_signature=(
        tf.TensorSpec(shape=(BATCH_SIZE, IMG_HEIGHT, IMG_WIDTH, 3), dtype=tf.float32),
        tf.TensorSpec(shape=(BATCH_SIZE, IMG_HEIGHT, IMG_WIDTH, 1), dtype=tf.float32)
    )
)

train_dataset = train_dataset.shuffle(buffer_size=100).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

# Custom Callback to Track Metrics
class MetricsCallback(tf.keras.callbacks.Callback):
    def __init__(self):
        self.dice_coeffs = []
        self.precisions = []
        self.recalls = []
        self.f1_scores = []

    def on_epoch_end(self, epoch, logs=None):
        Y_pred = (self.model.predict(X_val) > 0.5).astype(np.uint8)

        y_true_flat = Y_val.flatten()
        y_pred_flat = Y_pred.flatten()

        dice = dice_coefficient(Y_val, Y_pred).numpy()
        precision = precision_score(y_true_flat, y_pred_flat, zero_division=1)
        recall = recall_score(y_true_flat, y_pred_flat, zero_division=1)
        f1 = f1_score(y_true_flat, y_pred_flat, zero_division=1)

        self.dice_coeffs.append(dice)
        self.precisions.append(precision)
        self.recalls.append(recall)
        self.f1_scores.append(f1)

        print(f"\nEpoch {epoch+1} - Dice: {dice:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}")

# Train Model with Metrics Logging
metrics_callback = MetricsCallback()
history = model.fit(X_train,           # Training images
                    Y_train,
                    steps_per_epoch=len(X_train) // BATCH_SIZE,
                    epochs=EPOCHS,
                    validation_data=(X_val, Y_val),
                    callbacks=[metrics_callback])

# 📊 Plot Metrics Over Epochs
epochs_range = range(1, EPOCHS + 1)

plt.figure(figsize=(12, 6))

plt.subplot(2, 2, 1)
plt.plot(epochs_range, metrics_callback.dice_coeffs, 'b', label="Dice Coefficient")
plt.xlabel("Epochs")
plt.ylabel("Dice Coefficient")
plt.legend()

plt.subplot(2, 2, 2)
plt.plot(epochs_range, metrics_callback.precisions, 'g', label="Precision")
plt.xlabel("Epochs")
plt.ylabel("Precision")
plt.legend()

plt.subplot(2, 2, 3)
plt.plot(epochs_range, metrics_callback.recalls, 'r', label="Recall")
plt.xlabel("Epochs")
plt.ylabel("Recall")
plt.legend()

plt.subplot(2, 2, 4)
plt.plot(epochs_range, metrics_callback.f1_scores, 'm', label="F1 Score")
plt.xlabel("Epochs")
plt.ylabel("F1 Score")
plt.legend()

plt.suptitle("Training Metrics Over Epochs")
plt.show()


Epoch 1/50


/usr/local/lib/python3.11/dist-packages/keras/src/models/functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['keras_tensor']
Received: inputs=Tensor(shape=(None, 512, 512, 3))
  warnings.warn(msg)


63/70 ━━━━━━━━━━━━━━━━━━━━ 10s 1s/step - dice_coefficient: 0.1582 - loss: 0.8418  

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


UnknownError: Graph execution error:

Detected at node StatefulPartitionedCall defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "/usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py", line 37, in <module>

  File "/usr/local/lib/python3.11/dist-packages/traitlets/config/application.py", line 992, in launch_instance

  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelapp.py", line 712, in start

  File "/usr/local/lib/python3.11/dist-packages/tornado/platform/asyncio.py", line 205, in start

  File "/usr/lib/python3.11/asyncio/base_events.py", line 608, in run_forever

  File "/usr/lib/python3.11/asyncio/base_events.py", line 1936, in _run_once

  File "/usr/lib/python3.11/asyncio/events.py", line 84, in _run

  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelbase.py", line 510, in dispatch_queue

  File "/usr/local/lib/python3.11/dist-packages/ipyk